# Dynamic Asset Pricing

# Homework2

Ju Hyung Kang

## Using Monte Carlo

### (a)

The value of any (stochasctic) asset is the discounted expected value of its payoff. The payoff of a down-and-out call is

$$
(S_T-K)_{+} \mathbb{1}_{S_u \geq B, \forall u \in (t, T)}
$$

The discount rate should be the risk-free rate since $\mathbb{E}$ is in the risk neutral measure. Assuming the risk-free rate is constant from $t$ to $T$, the discount factor is $e^{-r(T-t)}$. Considering the information up to time $t$ ($\mathcal{F}_t$), the value of the option should be

$$
v_t=e^{-r(T-t)}\mathbb{E}[(S_T-K)_{+} \mathbb{1}_{S_u \geq B, \forall u \in (t, T)}|\mathcal{F}_t]
$$

### (b)

Note that

$$
S_{t+\Delta t}-S_t = rS_t \Delta t+\sigma S_t(W_{t+ \Delta t}-W_t)
$$

where $W_{t+ \Delta t}-W_t \sim N(0, \Delta t)$.

In [1]:
t = 0
T = 1
r = 0.02
sigma = 0.2
x = 100
B = 80
K = 110

In [2]:
def S_path(S0, r, sigma, t, T, N):
    path = np.zeros(N)
    path[0] = S0
    dt = (T-t)/N
    
    for i in range(1, N):
        S = path[i-1]
        
        path[i] = S + r*S*dt + sigma*S*np.random.normal(0, np.sqrt(dt))
        
    return path

In [3]:
import numpy as np

S_path(x, r, sigma, t, T, 252)

array([100.        , 100.23959733, 100.29234823, 101.0822914 ,
        99.78256239, 101.94567197, 102.73439171, 101.56391453,
       101.07921186, 101.94415122, 103.99814569, 103.68803373,
       105.21865238, 105.9438547 , 104.44746045, 106.56094415,
       106.03906254, 104.88737171, 104.20409833, 104.53830701,
       104.59453211, 104.67282464, 105.7171346 , 107.48353046,
       109.13257908, 110.11982364, 106.03267479, 105.35141392,
       102.32416048, 102.05594954, 100.47643491, 100.54627684,
       100.66127686, 102.02268377, 103.7019746 , 105.82877152,
       106.47108809, 104.58637164, 105.03329933, 106.13810961,
       106.7126972 , 108.46863367, 107.81921736, 105.61394498,
       108.5146667 , 109.30297855, 110.67032453, 109.10635906,
       107.69301732, 107.78855035, 107.13587496, 106.2732759 ,
       104.53822957, 105.53575367, 105.45087133, 106.69966137,
       107.14921546, 108.80745631, 105.72379214, 108.4944299 ,
       110.56704788, 109.45171175, 109.32483608, 107.89

### (c)

In [4]:
def down_and_out_call(path, K, B):
    if np.all(path > B):
        return max(path[-1] - K,0)
    else:
        return 0

In [5]:
down_and_out_call(S_path(x, r, sigma, t, T, 252), 110, 80)

0

### (d)

In [6]:
def simul(N_p, S0=100, r=0.02, sigma=0.2, t=0, T=1, N=252, B=80, K=110):
    val = 0
    for i in range(N_p):
        path = S_path(S0, r, sigma, t, T, N)
        val += down_and_out_call(path, K, B)
    
    return np.exp(-r*(T-t)) * val / N_p

In [7]:
price = simul(100000)

print(f"The price is {np.round(price, 2)}.")

The price is 4.91.


## Using the PDE

### (e)

Note that $\tau \leq T$. If $\tau < T$,

$$
\phi(y, \tau)=0.
$$

If $\tau = T$,

$$
\phi(y, \tau)=(y-K)_{+}
$$

The payoff of the down-and-out call given the stopping time is $\phi(y,s)$ where $s$ is the stopping time. This payoff function is equivalent to the payoff function given in (a). The value of any (stochasctic) asset is the discounted expected value of its payoff. Thus,

$$
v_t = \mathbb{E}[e^{-r(\tau-t)}\phi(S_\tau, \tau)|S_t=x]
$$

### (m)

In [21]:
t = 0
T = 1
r = 0.02
sigma = 0.2
B = 80
K = 110

In [22]:
R = 300
N_x = 2200
N_t = 252
dt = (T - t) / N_t
dx = (R - B) / N_x

In [23]:
from tqdm import tqdm

t = T
x = np.linspace(B, R, N_x + 1)
V = np.maximum(x - K, 0)

M = np.zeros((N_x + 1, N_x + 1))

M[0, 0] = 1
M[-1, -1] = 1

alpha = -dt * (r * x / (2 * dx) + (sigma**2) * (x**2) / (2 * (dx**2)))
beta = 1 + r * dt + (sigma**2) * dt / (dx**2) * (x**2)
gamma = -dt * (-r * x / (2 * dx) + (sigma**2) * (x**2) / (2 * (dx**2)))

np.fill_diagonal(M[1:-1, 2:], alpha)
np.fill_diagonal(M[1:-1, 1:], beta)
np.fill_diagonal(M[1:-1, 0:], gamma)

for i in tqdm(range(N_t, 0, -1)):
    C = np.append(np.zeros(N_x), np.exp(-r * (T - t + dt)) * K - np.exp(-r * (T - t)) * K)
    t -= dt
    
    V = np.linalg.inv(M) @ V + C

100%|████████████████████████████████████████████████████████████████████████████████| 252/252 [01:54<00:00,  2.21it/s]


In [24]:
# when initial price 100
x_idx = np.argmin(np.abs(x-100))

V_0 = V[x_idx]

In [25]:
V_0

4.908944853216507

### (n)

In [26]:
from sympy import symbols, Function, diff, pprint
from IPython.display import display

# Define the variable
x = symbols('x')
t = symbols('t')
B = symbols('B')
a = symbols('a')

q = Function('q')(x)
c = Function('c')(t, q)
f = Function('f')(t, x)

q = B**2 / x
f = (x / B)**(2*a) * c

f_t = diff(f, t)
f_x = diff(f, x)
f_xx = diff(f_x, x)

print("f_t")
display(f_t)
print("f_x")
display(f_x)
print("f_xx")
display(f_xx)

f_t


(x/B)**(2*a)*Derivative(c(t, q(x)), t)

f_x


2*a*(x/B)**(2*a)*c(t, q(x))/x + (x/B)**(2*a)*Derivative(c(t, q(x)), q(x))*Derivative(q(x), x)

f_xx


4*a**2*(x/B)**(2*a)*c(t, q(x))/x**2 + 4*a*(x/B)**(2*a)*Derivative(c(t, q(x)), q(x))*Derivative(q(x), x)/x - 2*a*(x/B)**(2*a)*c(t, q(x))/x**2 + (x/B)**(2*a)*Derivative(c(t, q(x)), q(x))*Derivative(q(x), (x, 2)) + (x/B)**(2*a)*Derivative(c(t, q(x)), (q(x), 2))*Derivative(q(x), x)**2

In [27]:
x = symbols('x')
t = symbols('t')
B = symbols('B')
r = symbols('r')
sigma = symbols('sigma')

# a = Function('a')(r, sigma)
a = symbols('a')
q = Function('q')(x)
c = Function('c')(t, q)
f = Function('f')(t, x)

# a = 1 / 2 * (1 - 2*r/sigma**2)
q = B**2 / x
f = (x / B)**(2*a) * c

f_t = diff(f, t)
f_x = diff(f, x)
f_xx = diff(f_x, x)

expression = -r*f + f_t + r*x*f_x + 1 / 2 * sigma**2 * x**2 * f_xx

display(expression)


r*x*(2*a*(x/B)**(2*a)*c(t, q(x))/x + (x/B)**(2*a)*Derivative(c(t, q(x)), q(x))*Derivative(q(x), x)) - r*(x/B)**(2*a)*c(t, q(x)) + 0.5*sigma**2*x**2*(4*a**2*(x/B)**(2*a)*c(t, q(x))/x**2 + 4*a*(x/B)**(2*a)*Derivative(c(t, q(x)), q(x))*Derivative(q(x), x)/x - 2*a*(x/B)**(2*a)*c(t, q(x))/x**2 + (x/B)**(2*a)*Derivative(c(t, q(x)), q(x))*Derivative(q(x), (x, 2)) + (x/B)**(2*a)*Derivative(c(t, q(x)), (q(x), 2))*Derivative(q(x), x)**2) + (x/B)**(2*a)*Derivative(c(t, q(x)), t)

### (o)

In [28]:
t = 0
T = 1
r = 0.02
sigma = 0.2
x = 100
B = 80
K = 110

In [29]:
# theo price
from scipy.stats import norm

def c(x, t=t, sigma=sigma, K=K, r=r, T=T):
    d1 = (np.log(x / K) + (r + 0.5 * (sigma**2)) * (T-t)) / (sigma * np.sqrt(T-t))
    d2 = d1 - sigma * np.sqrt(T-t)
    return x * norm.cdf(d1) - K * np.exp(-r * (T-t)) * norm.cdf(d2)

def f(x, B=B, t=t, sigma=sigma, K=K, r=r, T=T):
    alpha = 0.5 * (1 - 2 * r / sigma**2)
    return (x / B)**(2 * alpha) * c(B**2 / x, t, sigma, K, r, T)

v = c(x) - f(x)
v

4.920256808220372

#### Monte Carlo

- price: 4.908
- accuracy: discrete => not bad
- speed: slower


#### PDE

- price: 4.92
- accuracy: continuous => better
- speed: faster

## Bonus Question

By the put-call symmetry,
$$
C_K(T, x) = \sqrt{\frac{K}{H}}P_H(T, x)
$$

for $KH = F^2$ where $K, H$ are strikes of the call and put respectively and $F$ is the forward price. 

If the barrier has not been reached, the option value is equal to the payoff of the call with strike $K$.

If the barrier has been reached, we need to adjust the call payoff term at the moment when the barrier is touched.

Using the put-call symmetry with the forward price as the barrier $B$, the down-and-out barrier option can be replicated as follows:

$$
V(T, K, B, x) = C_K(T, x) - \frac{K}{B}P_{\frac{B^2}{K}}(T, x)
$$.

The put can also be replicated with a call and a forward.

In [30]:
t = 0
T = 1
r = 0
x = 100
B = 80
K = 80

for sigma in np.linspace(0.01, 3, 100):
    call = c(x, t=t, sigma=sigma, K=K, r=r, T=T)
    theo = call - f(x, B=B, t=t, sigma=sigma, K=K, r=r, T=T)
    
    rep_put = c(x, t=t, sigma=sigma, K=B**2 / K, r=r, T=T) - x + B**2 / K *np.exp(-r*(T))
    rep_ptf = call - K / B * rep_put
    
    print('sigma', np.round(sigma, 1), 'theo', np.round(theo, 1), 'rep_ptf', np.round(rep_ptf, 1))

sigma 0.0 theo 20.0 rep_ptf 20.0
sigma 0.0 theo 20.0 rep_ptf 20.0
sigma 0.1 theo 20.0 rep_ptf 20.0
sigma 0.1 theo 20.0 rep_ptf 20.0
sigma 0.1 theo 20.0 rep_ptf 20.0
sigma 0.2 theo 20.0 rep_ptf 20.0
sigma 0.2 theo 20.0 rep_ptf 20.0
sigma 0.2 theo 20.0 rep_ptf 20.0
sigma 0.3 theo 20.0 rep_ptf 20.0
sigma 0.3 theo 20.0 rep_ptf 20.0
sigma 0.3 theo 20.0 rep_ptf 20.0
sigma 0.3 theo 20.0 rep_ptf 20.0
sigma 0.4 theo 20.0 rep_ptf 20.0
sigma 0.4 theo 20.0 rep_ptf 20.0
sigma 0.4 theo 20.0 rep_ptf 20.0
sigma 0.5 theo 20.0 rep_ptf 20.0
sigma 0.5 theo 20.0 rep_ptf 20.0
sigma 0.5 theo 20.0 rep_ptf 20.0
sigma 0.6 theo 20.0 rep_ptf 20.0
sigma 0.6 theo 20.0 rep_ptf 20.0
sigma 0.6 theo 20.0 rep_ptf 20.0
sigma 0.6 theo 20.0 rep_ptf 20.0
sigma 0.7 theo 20.0 rep_ptf 20.0
sigma 0.7 theo 20.0 rep_ptf 20.0
sigma 0.7 theo 20.0 rep_ptf 20.0
sigma 0.8 theo 20.0 rep_ptf 20.0
sigma 0.8 theo 20.0 rep_ptf 20.0
sigma 0.8 theo 20.0 rep_ptf 20.0
sigma 0.9 theo 20.0 rep_ptf 20.0
sigma 0.9 theo 20.0 rep_ptf 20.0
sigma 0.9 